<img src="uva_seal.png">  

## Dimensionality Reduction

### University of Virginia
### DS 5110: Big Data Systems
### Last Updated: March 15, 2022

---  


### SOURCES
- Learning Spark, Chapter 11: Machine Learning with MLlib  
- https://spark.apache.org/docs/latest/mllib-dimensionality-reduction.html  
- https://github.com/apache/spark/blob/master/examples/src/main/python/mllib/pca_rowmatrix_example.py  
- https://github.com/apache/spark/blob/master/examples/src/main/python/mllib/svd_example.py  


### OBJECTIVES
-  Discuss purpose of dimensionality reduction  
-  Introduce Principal Component Analysis $(PCA)$  
-  Introduce Singular Value Decomposition $(SVD)$  


### CONCEPTS

- Matrix rank  
- Low-rank approximation of a matrix  
- Dimensionality reduction  
- $PCA$  
- $SVD$  

---

**Rank of a Matrix**

The **rank** of a matrix $A$ is the dimension of the vector space generated (or spanned) by its columns.  This corresponds to the maximal number of linearly independent columns of $A$.

The **column rank** of $A$ is the dimension of the column space of $A$.  

The **row rank** of $A$ is the dimension of the row space of $A$.  

A fundamental result in linear algebra is that the column rank and the row rank are always equal.


**Rank = 1**


\begin{equation*}
\mathbf{A} =  \begin{bmatrix}
1 & 1 & 0 & 2  \\
-1 & -1 & 0 & -2  \\
\end{bmatrix}
\end{equation*}

**Rank = 2**

\begin{equation*}
\mathbf{A} =  \begin{bmatrix}
1 & 0 & 1  \\
-2 & -3 & 1  \\
3 & 3 & 0  \\
\end{bmatrix}
\end{equation*}

**Dimensionality Reduction** 

For a matrix of data with (independent) features along the columns, the dimension is equal to the number of features under consideration.  

There are several reasons for reducing the number of dimensions including:  


- Visualization  


- $p >> n$ problem, or the curse of dimensionality (more features than observations)  
	Insufficient degrees of freedom to estimate a model


- Saving on storage requirements  
For example, a full matrix inversion step in regression (below) can be prohibitively large to store.


$$ \hat{β} = (X^TX)^{-1}X^TY $$


Instead, the Gram matrix $X^{T}X$ can be replaced by a lower rank matrix decomposition from $SVD$, as an example.

**Denoising the data** 

Even randomly generated data will produce a correlation matrix with extreme values by chance.  

Compressing information into a smaller set of features will reduce noise  

This is particularly useful when the covariance matrix is important 
(e.g., Mean Variance Portfolio Optimization in *Quantitative Finance*)






**Principal Component Analysis (PCA)**  

$PCA$ is the primary technique used for dimensionality reduction  

At a high level, $PCA$ constructs a set of new vectors which are a linear combination of the original vectors.  These new vectors have two special properties:  

1. The vectors form an orthogonal basis, which means they are uncorrelated  
2. The first principal component has the largest variance (it accounts for the most variability in the data), the second components has the next largest variance (while also being orthogonal to the first component), and so on.


**PCA Illustrated in Two Dimensions**  
The principal components point in the directions of greatest variance.  They are also orthogonal.





<img src="pca_img.gif">  

The number of principal components will be equal to the starting number of vectors. However, only a subset of the components will be used (the top $k$).  

**The top principal components can be used to create orthogonal, condensed features in a model.**  

**Pitfalls of PCA**  
$PCA$ includes a step where it computes the covariance matrix $X^{T}X$   
This can lead to numerical rounding errors when calculating the eigenvalues/vectors.  

For more details, please refer here for example:  
http://en.wikipedia.org/wiki/Principal_component_analysis  


**PCA Example (using RDD API)**

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
sc = SparkContext.getOrCreate()

rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/20 12:19:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
mat = RowMatrix(rows)

# Compute the top 4 principal components.
# Principal components are stored in a local dense matrix.
pc = mat.computePrincipalComponents(4)

23/06/20 12:19:29 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/06/20 12:19:29 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [3]:
print('dimensions of mat: {} rows, {} columns'.format(mat.numRows(), mat.numCols()))

dimensions of mat: 3 rows, 5 columns


In [4]:
print('dimensions of pc: {} rows, {} columns'.format(pc.numRows, pc.numCols))

dimensions of pc: 5 rows, 4 columns


In [5]:
# Project the rows to the linear space spanned by the top 4 principal components.
projected = mat.multiply(pc)
# Collect and print results
collected = projected.rows.collect()
print("Projected Row Matrix of principal component:")
for v in collected:
  print(v)

Projected Row Matrix of principal component:
[1.6485728230883814,-4.0132827005162985,-1.0091435193998504,-5.250587616986039]
[-4.645104331781532,-1.1167972663619048,-1.0091435193998501,-5.250587616986039]
[-6.428880535676488,-5.337951427775359,-1.009143519399851,-5.250587616986039]


In [6]:
# Aside: can convert sparse vector to dense vector as follows:
vs = Vectors.sparse(5, {1: 1.0, 3: 7.0})  # the sparse vector
vd = Vectors.dense(vs)

**PCA Example (using DataFrames API)**

In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors as dfVectors

spark = SparkSession.builder.getOrCreate()

# set up some data
data = [(dfVectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (dfVectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (dfVectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark.createDataFrame(data, ["features"])
df.show(truncate=False)

+---------------------+
|features             |
+---------------------+
|(5,[1,3],[1.0,7.0])  |
|[2.0,0.0,3.0,4.0,5.0]|
|[4.0,0.0,0.0,6.0,7.0]|
+---------------------+



**Notice the import line uses an alias dfVectors:**

`from pyspark.ml.linalg import Vectors as dfVectors`

We do this to avoid namespace collision with

`from pyspark.mllib.linalg import Vectors`

as both RDD and DF APIs have object *Vectors*

In [8]:
# PCA using 4 components
pca = PCA(k=4, inputCol="features", outputCol="pcaFeatures")

model = pca.fit(df)

# extract the transformed features
result = model.transform(df)

result.show(truncate=False)

+---------------------+-------------------------------------------------------------------------------+
|features             |pcaFeatures                                                                    |
+---------------------+-------------------------------------------------------------------------------+
|(5,[1,3],[1.0,7.0])  |[1.6485728230883814,-4.0132827005162985,-1.0091435193998504,-5.250587616986039]|
|[2.0,0.0,3.0,4.0,5.0]|[-4.645104331781533,-1.1167972663619048,-1.0091435193998504,-5.250587616986039]|
|[4.0,0.0,0.0,6.0,7.0]|[-6.428880535676488,-5.337951427775359,-1.0091435193998508,-5.250587616986039] |
+---------------------+-------------------------------------------------------------------------------+



**Singular Value Decomposition $(SVD)$**

$SVD$ is one of the major accomplishments of linear algebra, and it is a popular technique for factorizing an *m x n* (rectangular) matrix $A$ into three matrices with special properties.

\begin{equation*}
A   = U \Sigma V^T
\end{equation*}

$U$ is an orthonormal *m x m* matrix; its columns are called *left singular vectors*  
$\Sigma$ is a rectangular *m x n* diagonal matrix. It has nonnegative values in descending order  
$V$ is an orthonormal *n x n* matrix; its columns are called *right singular vectors*  

The diagonal entries of $\Sigma$ are known as the *singular values* of $A$.  
They are the square roots of the eigenvalues from the matrix $AA^T$

**<span style="color:red">The purpose of $SVD$ is to select only the top *k* singular values and their associated singular vectors.  </span>**

This means that we select only subsets of the matrices, which we denote as $\hat{U}$ , $\hat{\Sigma}$, and $\hat{V}^T$.  
An approximation to  $A$  can then be constructed as 

\begin{equation*}
\hat{A}   = \hat{U} \hat{\Sigma} \hat{V}^T
\end{equation*}

$\hat{U}$ has dimensions *m x k*  
$\hat{\Sigma}$  has dimensions *k x k*  
$\hat{V}^T$ has dimensions *k x n*  

The purpose of this factorization is to save on storage requirements, denoise, and recover the low-rank structure of the matrix.

For more details, please refer here for example:  
https://en.wikipedia.org/wiki/Singular_value_decomposition  

- If $n$ is small $(n<100)$ or $k$ is large compared with $n (k > n/2)$, we compute the Gramian matrix first and then compute its top eigenvalues and eigenvectors locally on the driver. This requires a single pass with $O(n^2)$ storage on each executor, and $O(n^2k)$ time on the driver. 


- Otherwise, we compute $(A^TA)v$ in a distributed way and send it to $ARPACK$ to compute $(A^TA)$’s top eigenvalues and eigenvectors on the driver node. This requires $O(k)$ passes, $O(n)$ storage on each executor, and $O(nk)$ storage on the driver.





**ARPACK**  
$ARPACK$ is a collection of `Fortran77` subroutines designed to solve large scale eigenvalue problems.

**$SVD$ Example**

In [9]:
from pyspark import SparkContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

sc = SparkContext.getOrCreate()

rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

mat = RowMatrix(rows)

# Compute the top k singular values and corresponding singular vectors.
topk = 4
svd = mat.computeSVD(topk, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

collected = U.rows.collect()
print("U factor is:")
for vector in collected:
    print(vector)
print("Singular values are: %s" % s)
print("V factor is:\n%s" % V)

U factor is:
[-0.38829130511665644,-0.9198099362554474,-0.056387441301709175,9.313225746154785e-09]
[-0.5301719995198351,0.2730185511901228,-0.8027319114319463,0.0]
[-0.7537556058139434,0.2817987790459642,0.5936682026454339,1.4901161193847656e-08]
Singular values are: [13.029275535600473,5.368578733451684,2.5330498218813755,6.323166049206486e-08]
V factor is:
DenseMatrix([[-0.31278534,  0.31167136,  0.30366911,  0.8409913 ],
             [-0.02980145, -0.17133211, -0.02226069,  0.14664984],
             [-0.12207248,  0.15256471, -0.95070998,  0.23828799],
             [-0.71847899, -0.68096285, -0.0172245 , -0.02094998],
             [-0.60841059,  0.62170723,  0.05606596, -0.46260933]])


**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) **PCA**  
i. Copy the PCA example code into the cell below  
ii. Define an RDD containing several vectors, each with length 5  
iii. Compute the principal components  
iv. Produce the scree plot and decide on the optimal number of components using the elbow method

2) **SVD**  
i. Copy the SVD example code into the cell below  
ii. Define an RDD using the same RDD from your PCA exercise  
iii. For singular values $k= 2, 3, 4$:  
- compute the approximation to the matrix  
- compute the Frobenius norm between the actual and approximate matrix, $||M_{act}-M_{approx}||_F$  
This matrix norm is the square root of the sum of the absolute squared element-wise differences between the matrices.  It measures a distance between two matrices.  

Hint: you may want to the convert data to numpy arrays.  Converting from RDD to numpy arrays can be done like this:

a = np.array(RDD.collect())

iv. What do you notice about the norm as $k$ increases?